In [ ]:
%pip install pwntools

In [ ]:
from pwn import remote
from Crypto.Util.number import getPrime
import json

In [ ]:
def json_recv(r):
    line = r.recvline()
    return json.loads(line.decode())

def json_send(r, hsh):
    request = json.dumps(hsh).encode()
    r.sendline(request)

In [ ]:
HOST = "socket.cryptohack.org"
PORT = 13403

r = remote(HOST, PORT)
line = r.recvline()
line

In [ ]:
q = int(line.split(b'"')[1], 16)
q

As we see in 13403.py, we will have to solve a discrete logarithm problem in order to find the private key. We see from the code we need to find an $n$ and $g$ such that $g^q = 1 \pmod n$.

This is a little bit tricky as we we'll need to choose a value of $n$ for which calculating the discrete logarithm is feasible. The first thing that comes to mind is to choose $n=q$, but then by Fermat's little theorem we to choose the trivial $g=1$, which will no allow us to find the private key $x$.

We can try $n=q^2$ for which we'll need to solve:
$$g^q = 1 \pmod {q^2}$$

We can use the interesting fact that:
$$(q+1)^a = \sum_{k=0}^a \binom{a}{k} q^k = 1 + aq + \sum_{k=2}^a \binom{a}{k} q^k = 1 + aq + q^2 \sum_{k=2}^a \binom{a}{k} q^{k-2}$$
Which means:
$$(q+1)^a = 1 + aq \pmod {q^2}$$
Or specifically:
$$(q+1)^q = 1 + q^2 = 1\pmod {q^2}$$
So by choosing $g=q+1$ we can find the private key $x$:
$$g^x = (q+1)^x = 1+xq \pmod {q^2} \Rightarrow x = \frac{g^x-1}{q}$$

In [ ]:
json_send(r, {'g':hex(q+1), 'n':hex(q**2)})
line = r.recvline()
line

In [ ]:
gx = int(line.split(b'"')[1], 16)
x = (gx - 1)//q
json_send(r, {'x':hex(x)})
line = r.recvline()
line